### This is a Training by DataTalksLab o Machine Learning Operations. For the entire End-To-End Production and Deployment , we will be using the New York Yello Taxi Dataset Data. This will be my maiden deployment.  ###

In [58]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import fastparquet as ParquetFile # The dataset is stored in parquet files 
import glob
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

In [2]:
#!pip install pyarrow


In [3]:
!pip install fastparquet # To read the files , we will need to install the fastparquet or pyarrow library

In [4]:
jan_df=pd.read_parquet("yellow_tripdata_2022-01.parquet")
feb_df=pd.read_parquet("yellow_tripdata_2022-02.parquet")
jan_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


# New Section

In [5]:
#prqtfiles=glob.glob("/home/hduser/Downloads/yellow_tripdata_2022-02.parquet" + "/*.parquet")
#for p in prqtfiles:
    #pr=ParquetFile(p,sep="\t")
    #df=pr.to_pandas()
#df.to_csv(p[:-8] + ".csv",index=False)
#df.head()

In [6]:
jan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [7]:
feb_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.0,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.0,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.5,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.0,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.5,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25


In [8]:
feb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2979431 entries, 0 to 2979430
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [9]:
print(jan_df.columns)
print("")
print(feb_df.columns)

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')


### Shape of the Data ###

In [10]:
print(jan_df.shape)
print("")
print(feb_df.shape)

(2463931, 19)

(2979431, 19)


In [11]:
jan_df.isnull().sum()

VendorID                     0
tpep_pickup_datetime         0
tpep_dropoff_datetime        0
passenger_count          71503
trip_distance                0
RatecodeID               71503
store_and_fwd_flag       71503
PULocationID                 0
DOLocationID                 0
payment_type                 0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
improvement_surcharge        0
total_amount                 0
congestion_surcharge     71503
airport_fee              71503
dtype: int64

In [12]:
feb_df.isnull().sum()

VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          101738
trip_distance                 0
RatecodeID               101738
store_and_fwd_flag       101738
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge     101738
airport_fee              101738
dtype: int64

In [13]:
jan_df['store_and_fwd_flag'].unique()
print('')
jan_df['store_and_fwd_flag'].value_counts()

N    2337496
Y      54932
Name: store_and_fwd_flag, dtype: int64

In [14]:
#Handling the missing values in the numerical columns by replacing them with the mean
missing_cols= ['passenger_count','RatecodeID','congestion_surcharge','airport_fee']
missing_cols

['passenger_count', 'RatecodeID', 'congestion_surcharge', 'airport_fee']

In [15]:
for col in missing_cols:
    mean_value = jan_df[col].mean()
    jan_df[col].fillna(mean_value,inplace=True)

In [16]:
for col in missing_cols:
    mean_value = feb_df[col].mean()
    feb_df[col].fillna(mean_value,inplace=True)

In [17]:
modal_val1=jan_df['store_and_fwd_flag'].mode()[0]
jan_df['store_and_fwd_flag'].fillna(modal_val1,inplace=True)

modal_val2=feb_df['store_and_fwd_flag'].mode()[0]
feb_df['store_and_fwd_flag'].fillna(modal_val2,inplace=True)

In [18]:
jan_df.isnull().sum()


VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
congestion_surcharge     0
airport_fee              0
dtype: int64

In [19]:
feb_df.isnull().sum()

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
congestion_surcharge     0
airport_fee              0
dtype: int64

In [20]:
print(jan_df.describe().T)
print("")
print(feb_df.describe().T)


                           count        mean         std     min     25%  \
VendorID               2463931.0    1.707819    0.502137    1.00    1.00   
passenger_count        2463931.0    1.389453    0.968601    0.00    1.00   
trip_distance          2463931.0    5.372751  547.871404    0.00    1.04   
RatecodeID             2463931.0    1.415507    5.831077    1.00    1.00   
PULocationID           2463931.0  166.076809   65.468057    1.00  132.00   
DOLocationID           2463931.0  163.580716   70.790159    1.00  113.00   
payment_type           2463931.0    1.194449    0.500178    0.00    1.00   
fare_amount            2463931.0   12.946484  255.814887 -480.00    6.50   
extra                  2463931.0    1.006710    1.236226   -4.50    0.00   
mta_tax                2463931.0    0.491454    0.082916   -0.50    0.50   
tip_amount             2463931.0    2.385696    2.830698 -125.22    0.72   
tolls_amount           2463931.0    0.374977    1.680538  -31.40    0.00   
improvement_

In [21]:
# Let us convert the Datasets to Csv files
#jan_df.to_csv('jan_df.csv',index=False)
#feb_df.to_csv('feb_df.csv',index=False)

In [22]:
#jan_df = jan_df.drop(jan_df([jan_df['Duration'] < 0].index, inplace=True))

In [23]:
print(jan_df.shape)
print(feb_df.shape)

(2463931, 19)
(2979431, 19)


In [24]:
# We first convert the ['tpep_dropoff_datetime'] and ['tpep_pickup_datetime'] to datetime format from Timedelta format
jan_df['tpep_dropoff_datetime']=pd.to_datetime(jan_df['tpep_dropoff_datetime'])
jan_df['tpep_pickup_datetime']=pd.to_datetime(jan_df['tpep_pickup_datetime'])
# We will do same for February ahead of applying the final model to it as our validation model
feb_df['tpep_dropoff_datetime']=pd.to_datetime(feb_df['tpep_dropoff_datetime'])
feb_df['tpep_pickup_datetime']=pd.to_datetime(feb_df['tpep_pickup_datetime'])

### Duration and Standard Deviation ###

In [25]:
#To compute a duration for the trip for both months
jan_df['Duration']=jan_df['tpep_dropoff_datetime'] - jan_df['tpep_pickup_datetime']
feb_df['Duration']=feb_df['tpep_dropoff_datetime'] - feb_df['tpep_pickup_datetime']
#jan_df['Duration'].dt.total_seconds()/60
jan_df['Duration']=jan_df['Duration'].apply(lambda td: td.total_seconds()/60)
feb_df['Duration']=feb_df['Duration'].apply(lambda td: td.total_seconds()/60)
print(jan_df.Duration.head())
print(feb_df.Duration.head())
#print("")
#print(feb_df.Duration.head())

0    17.816667
1     8.400000
2     8.966667
3    10.033333
4    37.533333
Name: Duration, dtype: float64
0    12.433333
1    17.550000
2    23.650000
3    20.083333
4    26.316667
Name: Duration, dtype: float64


In [26]:
#jan_df = jan_df[jan_df['Duration'] >= 0].dropna(how='any')

In [27]:
# Calculating the Standard Deviation
print(jan_df['Duration'].std())
#print(jan_df['Duration'].mean())

46.44530513776499


In [28]:
jan_ndf=jan_df.copy()

In [29]:
# Visualization of the Outliers for the Duration feature
#import seaborn as sns
#sns.lineplot(data=jan_df, x='Duration',y='trip_distance')

In [30]:
# Visualization of the Outliers for the Duration feature
#import seaborn as sns
#sns.boxplot(data=jan_df['Duration'])
#print("")
#sns.histplot(data=jan_df['Duration'])

### Dropping Outliers ###

In [31]:
# To remove the outliers below 1 minute and above 60 minutes for both months
mask_jan = jan_ndf[(jan_ndf['Duration']>= 1) & (jan_ndf['Duration']<= 60)]
jan_ndf=mask_jan
jan_ndf.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,37.533333


In [32]:
mask_feb = feb_df[(feb_df['Duration']>= 1) & (feb_df['Duration'] <= 60)]
feb_df=mask_feb
feb_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Duration
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.0,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25,12.433333
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.0,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25,17.550000
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.5,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25,23.650000
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.0,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00,20.083333
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.5,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25,26.316667


In [33]:
print(jan_ndf.shape)
print(feb_df.shape)

(2421440, 20)
(2918187, 20)


In [34]:
# The percentage of the dataset left after removing outliers in the dataset with duration less than 1 minute and less than 60minsless th
perc = len(jan_ndf)/len(jan_df)
perc

0.9827547930522406

**FEATURE ENGINEERING** 

In [35]:
#first we convert the features to datetime format
#jan_ndf['tpep_pickup_datetime']=pd.to_datetime(jan_ndf['tpep_pickup_datetime'])
#jan_ndf['tpep_dropoff_datetime']=pd.to_datetime(jan_ndf['tpep_dropoff_datetime'])

# We then extract relevant components and turn them to features like year,month,day,hour,minute and day)
#jan_ndf['dropoff_yr']=jan_ndf['tpep_dropoff_datetime'].dt.year
#jan_ndf['dropoff_mth']=jan_ndf['tpep_dropoff_datetime'].dt.month
#jan_ndf['dropoff_day']=jan_ndf['tpep_dropoff_datetime'].dt.day
#jan_ndf['dropoff_hour']=jan_ndf['tpep_dropoff_datetime'].dt.hour
#jan_ndf['dropoff_min']=jan_ndf['tpep_dropoff_datetime'].dt.minute

# We then extract relevant components and turn them to features like year,month,day,hour,minute and day)
#jan_ndf['pickup_yr']=jan_ndf['tpep_pickup_datetime'].dt.year
#jan_ndf['pickup_mth']=jan_ndf['tpep_pickup_datetime'].dt.month
#jan_ndf['pickup_day']=jan_ndf['tpep_pickup_datetime'].dt.day
#jan_ndf['pickup_hour']=jan_ndf['tpep_pickup_datetime'].dt.hour
#jan_ndf['pickup_min']=jan_ndf['tpep_pickup_datetime'].dt.minute



# Using the sklearn's get_dummies we then apply a one_hot encoding to the new features

#df_encoded=pd.get_dummies (jan_ndf, columns=['dropoff_yr','dropoff_mth','dropoff_day','dropoff_hour','dropoff_min','pickup_yr',
#'pickup_mth','pickup_day','pickup_hour','pickup_min'])




In [36]:
jan_ndf['Duration'].head()

0    17.816667
1     8.400000
2     8.966667
3    10.033333
4    37.533333
Name: Duration, dtype: float64

In [37]:
#jan_ndf.drop(['tpep_pickup_datetime','tpep_dropoff_datetime'],axis=1,inplace=True)

In [38]:
select_cols= ['PULocationID','DOLocationID']
select_cols

['PULocationID', 'DOLocationID']

In [39]:
jan_ndf[select_cols]=jan_ndf[select_cols].astype(str)

In [40]:
jan_ndf[select_cols].head()

,PULocationID,DOLocationID
0,142,236
1,236,42
2,166,166
3,114,68
4,68,163


### Dimensionality Matrix ###

In [41]:


#To convert dataframe into a list of dictionaries
df_list = jan_ndf[select_cols].to_dict('records')

# Initialize Dictionary Vectorizer
vectorizer=DictVectorizer()

#fit the vectorizer on the data_lists to obtain the feature matrix
train_df = vectorizer.fit_transform(df_list)

# To obtain the dimensionality of the feature matix
jan_num_features=train_df.shape[1]

print("Dimensionality of the Feature Matrix:", jan_num_features)



Dimensionality of the Feature Matrix: 515


**Model**

In [42]:
X_train= train_df
y_train= jan_ndf['Duration'].values



In [43]:
X_train


<2421440x515 sparse matrix of type '<class 'numpy.float64'>'
	with 4842880 stored elements in Compressed Sparse Row format>

In [44]:
y_train

array([17.81666667,  8.4       ,  8.96666667, ..., 11.        ,
       12.05      , 27.        ])

In [45]:
lr = LinearRegression()



In [46]:
lr.fit(X_train,y_train)

LinearRegression()

In [47]:
y_pred=lr.predict(X_train)

In [48]:
mse=mean_squared_error(y_train,y_pred)
mse

48.80686240368459

In [49]:

rmse= np.sqrt(mse)
rmse

6.986190836477672

In [50]:
# The validation set
val_cols= ['PULocationID','DOLocationID']
valdf=feb_df[val_cols]
valdf

,PULocationID,DOLocationID
0,138,252
1,138,41
2,138,200
3,239,200
4,138,125
...,...,...
2979426,163,193
2979427,141,4
2979428,161,151
2979429,141,226


In [51]:
val_cols=valdf.astype(str)

In [52]:


#To convert dataframe into a list of dictionaries
feb_df_list = val_cols.to_dict("records")


#fit the vectorizer on the data_lists to obtain the feature matrix
val_df = vectorizer.transform(feb_df_list)



In [53]:
X_val= val_df
y_val= feb_df['Duration'].values


In [54]:
y_pred_val=lr.predict(X_val)

In [55]:
mse=mean_squared_error(y_val,y_pred_val)
mse

60.6281618475893

In [56]:
rmse= np.sqrt(mse)
rmse

7.78640879016696

In [59]:
#from sklearn.externals import joblib
# Save the model to a file
#joblib.dump(model, 'linear_regression_model.pkl')

# Load the model
#loaded_model = joblib.load('linear_regression_model.pkl')


# Save the model to a file
with open('lr_model.pkl', 'wb') as file:
    pickle.dump(lr, file)

In [ ]:
# Lets load the saved model

with open('lr_model.pkl', 'rb') as file:
    #loaded_model = pickle.load(file)
